In [105]:
import requests 
import re as regex
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import jupyter_util as util

## Enter Invite Url:

In [106]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNGM5MGY5YWYtNmRiOS00ZDFiLTlhOGYtNDQ1ZTFmMjlhMTU4IiwgInJlY2lwaWVudEtleXMiOiBbIkZTaGpMdGNzakJZZDY1VFk4UkhKbUp3NGNBWGV1YXB3cUJKOThnUTFSRXRwIl0sICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovL2FnZW50MS5sb2NhbGhvc3QiLCAibGFiZWwiOiAiTWFpbiAoYWRtaW4pIn0=


In [107]:
# Process invite url, delete white spaces
agent1['url'] = agent1['url'].replace(" ", "")
# Regex(substitution) to extract only the invite string from url
agent1['invite_string_b64'] = regex.sub(
           r".*(c\_i\=)", 
           r"", 
           agent1['url'])
# Decoding invite string using base64 decoder
agent1['invite_string'] = base64.b64decode(agent1['invite_string_b64'])
# Converting our invite json string into a dictionary 
agent1['invite'] = json.loads(agent1['invite_string'])

print("Decoded invitation:\n")
pp.pprint(agent1['invite'])

Decoded invitation:

{'@id': '4c90f9af-6db9-4d1b-9a8f-445e1f29a158',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Main (admin)',
 'recipientKeys': ['FShjLtcsjBYd65TY8RHJmJw4cAXeuapwqBJ98gQ1REtp'],
 'serviceEndpoint': 'http://agent1.localhost'}


In [108]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

# print(base58.b58encode(did))
# print(plugin['did'])
print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 GPGR1z94KnZQUgRyNeu1Pm3UmStZJhyyUPyEkNLxRkCcjjyk33fcn8r1Pu6BvFe1Ko2jMYt4d1gkMioyhG14Tfe

Public key: 
 AUPLa4sYhmQHjp5EnLDE35B8RtXBCaMYksVT4iBWsnZE


**Sending a connection request to acapy**

In [109]:
uniqueId = str(uuid.uuid4())
# print(uniqueId)

# our request body
message = {
        "@id":  uniqueId,
        "~transport": {
          "return_route": "all"
        },
        "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/request",
        "label": "Plugin",
        "connection": {
          "DID": plugin['did'],
          "DIDDoc": {
            "@context": "https://w3id.org/did/v1",
            "id": plugin['did'],
            "publicKey": [{
              "id": plugin['did'] + "#keys-1",
              "type": "Ed25519VerificationKey2018",
              "controller": plugin['did'],
              "publicKeyBase58": plugin['public_key_b58']
            }],
            "service": [{
              "id": plugin['did'] + ";indy",
              "type": "IndyAgent",
              "recipientKeys": plugin['public_key_b58'],
              "serviceEndpoint": ""
            }]
          }
        }
      }

**Encoding/packing the request**

In [110]:
# encoding it with aries crypto function using the key that was
# given to us by aca-py in recipientKeys
decodedAcapyKey = base58.b58decode(agent1['invite']['recipientKeys'][0])
ourPrivateKey = plugin['keypair'][1]
# print(plugin['did'])
encodedMessage = \
    crypto.encode_pack_message(json.dumps(message), [decodedAcapyKey], ourPrivateKey)

encodedMessage = encodedMessage.decode("ascii")

In [111]:
connectionRequestResponse = requests.post(agent1['invite']['serviceEndpoint'], data=encodedMessage)
assert connectionRequestResponse.text != "", "invalid response from acapy"

**Response from aca-py**

In [112]:
   
connectionRequestResponseUnpacked = \
    util.unpackMessage(connectionRequestResponse.text, plugin['keypair'][1])

connectionRequestResponseDict = json.loads(connectionRequestResponseUnpacked[0])

print("Decoded acapy response: \n")
pp.pprint(connectionRequestResponseUnpacked)

Decoded acapy response: 

('{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/response", '
 '"@id": "92575ebb-f835-43c3-bbd8-efaea4dc5688", "~thread": {"thid": '
 '"164be4b3-72fc-4da3-a6e0-b8aef784ed59"}, "connection~sig": {"@type": '
 '"did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/signature/1.0/ed25519Sha512_single", '
 '"signature": '
 '"G7cQ4olnbGkQ-nRjYjfam8DeniXt923T3Kp4UnW2ycBSZ8R3EiBJ5mh4tPgBxtPElvjy6M6FQWfF9W8CUpEgCA==", '
 '"sig_data": '
 '"AAAAAF7DyDF7IkRJRCI6ICI5SnZlYWFwdXlwb05ZM0hSVTNYOW1qIiwgIkRJRERvYyI6IHsiQGNvbnRleHQiOiAiaHR0cHM6Ly93M2lkLm9yZy9kaWQvdjEiLCAiaWQiOiAiZGlkOnNvdjo5SnZlYWFwdXlwb05ZM0hSVTNYOW1qIiwgInB1YmxpY0tleSI6IFt7ImlkIjogImRpZDpzb3Y6OUp2ZWFhcHV5cG9OWTNIUlUzWDltaiMxIiwgInR5cGUiOiAiRWQyNTUxOVZlcmlmaWNhdGlvbktleTIwMTgiLCAiY29udHJvbGxlciI6ICJkaWQ6c292OjlKdmVhYXB1eXBvTlkzSFJVM1g5bWoiLCAicHVibGljS2V5QmFzZTU4IjogIjVYZmJoRnJQeHRCNkt6S1ZRYm4ybkE5b2tOZnZLR1JjdDd1aUJYRkZKdWNZIn1dLCAiYXV0aGVudGljYXRpb24iOiBbeyJ0eXBlIjogIkVkMjU1MTlTaWduYXR1cmVBdXRoZW50aWNhdGlvbjIwMT

**Unpacking connection data embedded in the response**

In [113]:
sig_data_raw = connectionRequestResponseDict['connection~sig']['sig_data']
#  (this is a hack)replacing artifacts that sometimes happen
sig_data_raw = sig_data_raw.replace("-", "1")
sig_data_raw = sig_data_raw.replace("_", "1")

sig_data_raw = base64.b64decode(sig_data_raw)
# avoid first 8 characters as they are a time signature
sig_data_raw = sig_data_raw[8:]
sig_data_raw = sig_data_raw.decode('ascii')

sig_data = json.loads(sig_data_raw)
pp.pprint(sig_data)

{'DID': '9JveaapuypoNY3HRU3X9mj',
 'DIDDoc': {'@context': 'https://w3id.org/did/v1',
            'authentication': [{'publicKey': 'did:sov:9JveaapuypoNY3HRU3X9mj#1',
                                'type': 'Ed25519SignatureAuthentication2018'}],
            'id': 'did:sov:9JveaapuypoNY3HRU3X9mj',
            'publicKey': [{'controller': 'did:sov:9JveaapuypoNY3HRU3X9mj',
                           'id': 'did:sov:9JveaapuypoNY3HRU3X9mj#1',
                           'publicKeyBase58': '5XfbhFrPxtB6KzKVQbn2nA9okNfvKGRct7uiBXFFJucY',
                           'type': 'Ed25519VerificationKey2018'}],
            'service': [{'id': 'did:sov:9JveaapuypoNY3HRU3X9mj;indy',
                         'priority': 0,
                         'recipientKeys': ['5XfbhFrPxtB6KzKVQbn2nA9okNfvKGRct7uiBXFFJucY'],
                         'serviceEndpoint': 'http://agent1.localhost',
                         'type': 'IndyAgent'}]}}


**Adding the connection to connection list**

In [114]:
connections = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": sig_data['DIDDoc'],
        "myKey": plugin['keypair'][1]
    }
]

pp.pprint(connections)

[{'DIDDoc': {'@context': 'https://w3id.org/did/v1',
             'authentication': [{'publicKey': 'did:sov:9JveaapuypoNY3HRU3X9mj#1',
                                 'type': 'Ed25519SignatureAuthentication2018'}],
             'id': 'did:sov:9JveaapuypoNY3HRU3X9mj',
             'publicKey': [{'controller': 'did:sov:9JveaapuypoNY3HRU3X9mj',
                            'id': 'did:sov:9JveaapuypoNY3HRU3X9mj#1',
                            'publicKeyBase58': '5XfbhFrPxtB6KzKVQbn2nA9okNfvKGRct7uiBXFFJucY',
                            'type': 'Ed25519VerificationKey2018'}],
             'service': [{'id': 'did:sov:9JveaapuypoNY3HRU3X9mj;indy',
                          'priority': 0,
                          'recipientKeys': ['5XfbhFrPxtB6KzKVQbn2nA9okNfvKGRct7uiBXFFJucY'],
                          'serviceEndpoint': 'http://agent1.localhost',
                          'type': 'IndyAgent'}]},
  'label': 'Main (admin)',
  'myKey': b"\rD8Z\x1d\xb1\x90)P\x18\xafi\xc5l\xf1\r\xabW1]\xb1\\\xea

### FEATURE DISCOVERY

In [115]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/query",
    query="*"
)
message = util.packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=message)
# print(response.text)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '4ac8247a-34b4-4572-a04b-38eaaaa57973',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/disclose',
 'protocols': [{'pid': 'https://didcomm.org/credential-issuance/0.1'},
               {'pid': 'https://didcomm.org/notification/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/routing/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/credential-presentation/0.1'},
               {'pid': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-payments/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/credential-issuance/0.1'},
               {'pid': 'https://didcomm.org/trust_ping/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof

## Dids

In [116]:
message = util.buildMessage("did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-list-dids")
encodedMessage = util.packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '4be65c49-d894-4b82-b0ab-321a3ecf0e88',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/list-dids',
 'result': [{'did': 'WTSnQMUdQjdsxYkKUBTcYJ',
             'verkey': 'H4AJNxyBHdmgENQsME6dhXS345NLVqcvvPM7QtF2X3b5'},
            {'did': 'WjVuoNs4Zc1mK6vXPfqnj5',
             'verkey': 'HCuoA2ewtHWQ297KgJG22VrexYmEbZRCi4aqPkE2cEt9'},
            {'did': 'CcPNApHoaNaxHtrDJx3Fcu',
             'verkey': '7L23hAKHoiLQ4KyaTEkpzi5eoLXt3cLuteFYp6CzDxXk'},
            {'did': 'AH7bQLKyijZkniHvRoLAzu',
             'verkey': '64HqNtZaju4Y5Vd4ZzzBZ4apZqe69gc6AtF9zHr5S9dr'},
            {'did': 'TUNT4Z7kKeYquhmeCZjg8F',
             'verkey': 'FRq4sMiB64yErvd4Rs7k4dmKxzBBVkbFh4Rt6QGmXmbZ'},
            {'did': 'B2xyyghsCnQ2svVzD6uce5',
             'verkey': '6UC4vdz81bf5ZCgQk2DoQho3d8jMxmsRbC5StZHHwtc4'},
            {'did': 'WZndzdvhBc6vXp8hFPrCdL',
             'verkey': 'H7co14Vem8cv3BQ4GLgu3yutTHRRikXfUFofspX9vpDd'},
            {'did': 'qREBhbMGwvNYXYVehH82e',
       

## Public did

In [117]:
message = util.buildMessage("did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-public-did")
encodedMessage = util.packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '5cf29f59-e1eb-4125-baff-72b1633f9ecc',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 '~thread': {'thid': 'b5a024e2-3bef-4a99-83e1-22a558c907a4'}}


In [120]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/schema",
    payload="a very cool message"
)

encodedMessage = util.packMessage(message, connections[0])
# print(encodedMessage)

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)
# print(response.text)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '4357ba48-034f-4f03-8d65-9bd2ef47bb03',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/schema',
 'payload': 'a very cool message',
 'schema_id': '6f1b20eaa0a144e6aa5744fe6eaabede',
 '~thread': {'thid': 'a30634d9-a264-4d9c-940b-d0a7a179afb9'}}


In [121]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/search_schema",
    schema_id="6f1b20eaa0a144e6aa5744fe6eaabede"
)

encodedMessage = util.packMessage(message, connections[0])

endpoint = connections[0]['DIDDoc']['service'][0]['serviceEndpoint']
response = requests.post(endpoint, data=encodedMessage)

responseDecoded = util.unpackMessage(response.text, plugin['keypair'][1])
responseDecoded = json.loads(responseDecoded[0])
pp.pprint(responseDecoded)

{'@id': '2bb348f5-9d55-448e-b6c0-fc225c010611',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/search_schema',
 'schema_id': '6f1b20eaa0a144e6aa5744fe6eaabede',
 '~thread': {'thid': '22eb5f23-e4f4-4e98-bc92-e65da0d622fe'}}
